In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display

# Process Data

In [ ]:
#!/home/lgunthermann/miniconda3/envs/discushack22/bin/python3 process_csv.py

In [ ]:
data_coding = pd.read_csv('data/processed/coding_both.csv')
data_invoice = pd.read_csv('data/processed/invoices.csv')

In [ ]:
df = pd.DataFrame().reindex(columns=data_coding.columns)
df = df.rename(columns={"Expense Category": "Expense Category (Coding)"})
df.insert(1,'ECRef',[])
df.insert(6,'Expense Category (Invoice)',[])
df.drop(['ID'], axis=1)

# Output Dataframe

The new dataframe consists of 6 columns. 
- ECRef (linking the entry to the invoice list)
- InvoiceID (ID of the invoice copy)
- Supplier
- Party
- Total Spend
- Error (Mismatch in total amounts, to identify invoices for manual annotation)
- Expense Category (According to the coding list)
- Expense Category (According to the invoice list)
- 58 expense categorisations

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def print_positions(party,supplier):
    print("Invoices:")
    invoices = data_invoice.loc[ (data_invoice['RegulatedEntityName'] == party) & (data_invoice['SupplierName'] == supplier) ]
    display(invoices)
    print("Sum TotalExpenditure:",invoices['TotalExpenditure'].sum())
    print("\nCoding:")
    display(data_coding.loc[ (data_coding['Party'] == party) & (data_coding['Supplier'] == supplier) ])
    
def print_supplier(supplier):
    print("Invoices:")
    invoices = data_invoice.loc[ (data_invoice['SupplierName'] == supplier) ]
    display(invoices)
    print("Sum TotalExpenditure:",invoices['TotalExpenditure'].sum())
    print("\nCoding:")
    display(data_coding.loc[ (data_coding['Supplier'] == supplier) ])
    
def get_nth_key(dictionary, n=0):
    if n < 0:
        n += len(dictionary)
    for i, key in enumerate(dictionary.keys()):
        if i == n:
            return key
    raise IndexError("dictionary index out of range") 

In [ ]:
#display(data_coding)

# Data Preview

In [ ]:
data_coding.head()

In [ ]:
data_invoice.head()

# Solver

In [ ]:
def get_totals(invoice_amounts,total_values,prediction):
    totals = np.zeros_like(total_values)
    for amount, cat in zip(invoice_amounts,prediction):
        totals[cat] += amount
    return totals

def get_error(invoice_amounts,total_values,prediction,loss='MSE',return_totals=False,return_diff=False):
    diff = total_values - get_totals(invoice_amounts,total_values,prediction)
    if loss == 'MSE':
        error = np.square(diff)
    else:
        error = np.abs(diff)
        
    r = [np.mean(error)]
    if return_totals: r+= [totals]
    if return_diff: r+= [diff]
    return r

In [ ]:
def random_solver(invoice_amounts,total_values,loss='MSE',max_iter=5,max_waiting_iter=1):
    factor = invoice_amounts.shape[0] ** (len(total_values)-1)
    max_iter = int(max_iter * factor)
    max_waiting_iter = int(max_waiting_iter * factor)
    pred = np.random.randint(low=0, high=len(total_values), size=invoice_amounts.shape[0], dtype=int)

    error, diff = get_error(invoice_amounts,total_values,pred,loss,return_diff=True)
    wait = 0
    unvisited = list(range(len(pred)))
    while len(unvisited) > 0:   
        idx = unvisited.pop(0)
        cat = pred[idx]
        
        #print(idx,unvisited)
        
        error_, diff_, cat_ = error, diff, cat
        for c in range(len(total_values)):
            if c==cat: continue
            pred[idx] = c
            e, d = get_error(invoice_amounts,total_values,pred,loss,return_diff=True)
            #print(f"{c=} {e=}")
            if e<error_:
                error_, diff_, cat_ = e, d, c
                
        if error_ < error:
            unvisited = list(range(len(pred)))
            np.random.shuffle(unvisited)
            #print(f"Improvement! {error_}")

                
        pred[idx] = cat_
        error, diff = error_, diff_
            
    return pred

In [ ]:
def genetic_algorithm(invoice_amounts,total_values,loss='MSE',N=100,max_iter=5,mutationRate=0.02):
    No_Gen = invoice_amounts.shape[0]
    No_Typ = len(total_values)
    
    factor = No_Gen ** (No_Typ-1)
    max_iter = int(max_iter * factor)
    
    indeces = np.array([range(N)])
    pop = np.random.randint(low=0, high=No_Typ, size=(N,No_Gen), dtype=int)
    fit = np.array([get_error(invoice_amounts,total_values,pred,loss=loss)[0] for pred in pop])
    
    for it in range(max_iter):
        champs = np.random.choice(indeces, size=2, replace=False)
        
        # determine winner
        if fit[champs[1]] < fit[champs[0]]:
            champs[0],champs[1] = champs[1],champs[0]
            
        # uniform crossover
        for j in range(No_Gen):
            # Mutation chance
            if random.random() < mutationRate:
                pop[champs[1],j] = np.random.randint(low=0, high=No_Typ, size=1, dtype=int)
            
            # 50% chance of overwritting
            elif random.random() > 0.5:
                pop[champs[1],j] = pop[champs[0],j] * ( 1 - 2 * int(round(random.random()-0.5+mutationRate)))


        # determine fitness of newly combined individual
        fit[champs[1]] = calcFit(pop[champs[1]])

In [ ]:
parties = pd.concat((data_coding['Party'], data_invoice['RegulatedEntityName'])).unique()
categories = data_coding.columns[5:]

error_acc = 0
row_list = []

for party in parties:
    party_coding = data_coding[ data_coding['Party']==party ]
    party_invoice = data_invoice[ data_invoice['RegulatedEntityName']==party ]
    
    suppliers = pd.concat(( party_coding['Supplier'], party_invoice['SupplierName'] )).unique()
    
    for supplier in suppliers:
        sup_coding = party_coding[ party_coding['Supplier']==supplier ]
        sup_invoice = party_invoice[ party_invoice['SupplierName']==supplier ]
        
        sup_cats = {}
        
        for cat in categories:
            val = np.sum(sup_coding[cat].values)
            if val >= 0.01:
                sup_cats[cat] = val
        
        if len(sup_cats) < 1:
            continue
        
        spent = np.round(sup_coding['Total Spend'].sum(),2)
        filed = np.round(sup_invoice['TotalExpenditure'].sum(),2)
        
        print(f"{party} - {supplier}")
        print(f"Diff = {abs(spent-filed)} ({spent=} | {filed=}) accross {len(sup_cats)} categories with {sup_coding.shape[0]} codings and {sup_invoice.shape[0]} invoices.")
        
        # Estimate a proper matching if there's more than one invoice
        # random encoding which invoice belongs to which category
        if sup_invoice.shape[0] > 1:
            values = np.fromiter(sup_cats.values(), dtype=float)
            invoices = sup_invoice['TotalExpenditure'].values
            
            if len(values) <= 1:
                prediction = np.zeros(invoices.shape[0],dtype=int)
            else:
                # Absolute error is fine for random solver as only one position is changed at the time
                prediction = random_solver(invoices,values,loss='MAE')
                
                
            
            # get non-squared error
            error = get_error(invoices,values,prediction,loss='MAE')[0]
            error_acc += error

            for i, (idx, row) in enumerate(sup_invoice.iterrows()):
                val = row['TotalExpenditure']
                entry = {
                    'ECRef': row['ECRef'],
                    'InvoiceID': row['RedactedSupportingInvoiceId'],
                    'Supplier': supplier,
                    'Party': party,
                    'Total Spend': val,
                    'Error': error,
                    'Expense Category (Coding)': sup_coding['Expense Category'].mode()[0],
                    'Expense Category (Invoice)': row['ExpenseCategoryName'],
                    **{k:0 for k in categories}
                        }

                cat = get_nth_key(sup_cats, n=prediction[i])
                entry[cat]=val

                row_list.append(entry)
        else:
            row = sup_invoice.iloc[0]
            entry = {
                'ECRef': row['ECRef'],
                'InvoiceID': row['RedactedSupportingInvoiceId'],
                'Supplier': supplier,
                'Party': party,
                'Total Spend': row['TotalExpenditure'],
                'Error': 0.0,
                'Expense Category (Coding)': sup_coding['Expense Category'].mode()[0],
                'Expense Category (Invoice)': row['ExpenseCategoryName'],
                **{k:0 for k in categories}
                    }
            entry.update(sup_cats)
            row_list.append(entry)
            
            
        if False:
            print(f"{party} - {supplier}")
            print(f"Diff = {abs(spent-filed)} ({spent=} | {filed=}) accross {len(sup_cats)} categories with {sup_coding.shape[0]} codings and {sup_invoice.shape[0]} invoices.")

            print("\nCoding:")
            display(sup_coding)

            print("\nInvoice:")
            display(sup_invoice)
            
df = pd.DataFrame.from_dict(row_list)

In [ ]:
print("Error:",error_acc)
df.head()

In [ ]:
df.to_csv('data/processed/invoices_matched.csv', index=False)

In [ ]:
df = df.sort_values(by=['Error'], ascending=False)
df.head()

In [ ]:
print_positions(party='Labour Party',supplier='Experian')